$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{Universidad de los Andes}$$
$$\textrm{Grupo de Física de Altas Energías: Fenomenología de Partículas}$$

Este código fue corrido en el docker del grupo de fenomenología.

$\textbf{Preliminares}$ 

Las librerías que se usan en este capítulo son las siguientes: 

In [1]:
import os, sys
import numpy as np

personal_folder = os.getcwd()

$\textbf{Crear la carpeta del output de MadGraph}$ 

Inicialmente es necesario crear la carpeta donde se guardarán los datos (outputs) del proceso, para esto hay que escribir un script con las siguientes líneas: 

- generate p p > z > mu+ mu- 

- output /disco4/personal_folders/Joaquin/Data_Z -no-jpeg 

Esto se podría hacer como siempre, sin embargo, es una buena oportunidad para escribir scripts desde python y tratar de hacer todo desde acá: 

In [2]:
lineas_definiciones = ["generate p p > z > mu+ mu-"]
lineas_outputs = [f"output {personal_folder}/Data_Z -no-jpeg"]

lineas = np.concatenate((lineas_definiciones, lineas_outputs))

script = open ('Folder_Data_Z.txt','w')
script.write(lineas[0])

for i in range(1,len(lineas)):
    script.write('\n' + lineas[i])
script.close()

Veamos que efectivamente se creó el script que queríamos:

In [3]:
!cat Folder_Data_Z.txt

generate p p > z > mu+ mu-
output /disco4/personal_folders/Joaquin/Data_Z -no-jpeg

Ahora es necesario correr el script, para hacerlo desde jupyter notebook hay que usar el siguiente comando:

In [ ]:
!./../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC Folder_Data_Z.txt
!rm Folder_Data_Z.txt #Borramos el script porque ya lo usamos

$\textbf{Generar eventos usando MadGraph}$

Supongamos que queremos generar 300k de eventos con algunos parámetros, normalmente se tendrían que hacer varios scripts para generarlos en grupos de a 50K; sin embargo, otra forma es haciendo los scripts desde python y usar un for, eso es lo que haremos. 

Sin embargo, es importante crear una función que nos permita usar una semilla distinta en cada run del proceso:

In [ ]:
seeds = [] #Arreglo donde guardaremos todas las semillas

def semilla():
    seed = np.random.randint(1,5000)
    if (seed in seeds):
        return semilla()
    return seed

In [ ]:
for k in range(0,6): #Se corre 6 veces para garantizar que hayan 6 "runs" con 50000 eventos, es decir, 300000 eventos  
    
    #Primero calculemos la semilla que usaremos en este run:
    seed = semilla()
    seeds.append(seed)
    
    #Ahora creemos un script temporal para correrlo con MadGraph    
    nombre_script = 'generate_data_z.txt'  
    script = open (nombre_script,'w')

    launch = f"launch {personal_folder}/Data_Z -m"
    nodos = "10"
    shower = "shower = Pythia8"
    detector = "detector = Delphes"
    done = "done"
    param1 = "set iseed " + str(seed) #Semilla
    param2 = "set nevents 50000"

    lineas = [launch,nodos,shower,detector,done,param1, param2]

    script.write(lineas[0])
    for k in range(1,len(lineas)):    
        script.write('\n' + lineas[k])
    script.close()

    !./../../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC {nombre_script} #Se corre el script temporal con MadGraph para generar los datos
    !rm {nombre_script} #Se borra el script temporal

Es importante dejar un archivo como testigo que permita saber cuándo el código de arriba ya acabo de correr, así nos aseguramos de que todo el código se corra adecuadamente: 

In [ ]:
!touch Data_Z_generada.txt 

En este punto ya se simuló la señal que se quiere estudiar, ahora uno podría pensar en generar datos para el background; sin embargo, para propósitos de este tutorial usaremos datos de background que ya están en el servidor. 